# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [19]:
%pip install torchmetrics clu ml-collections torch_geometric polars rdkit -q

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install -U "ray[tune]" -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

In [ ]:
%pip show jax | grep Version

In [ ]:
!nvidia-smi

## PNA EPCSAFT

In [ ]:
import wandb
wandb.login()

In [21]:
%cd GNN-ePC-SAFT

[Errno 2] No such file or directory: 'GNN-ePC-SAFT'
/notebooks/GNN-ePC-SAFT


In [ ]:
!python train.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7 \
--config.checkpoint_every_steps=2000 --config.learning_rate=0.0001 --config.patience=1000 \
--config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128 \
--config.propagation_depth=4 --config.hidden_dim=64 --config.num_mlp_layers=2 --config.pre_layers=1 --config.post_layers=3

In [23]:
!python tuner.py --workdir=/notebooks/GNN-ePC-SAFT/ --config=configs/default.py \
--config.num_train_steps=10000 --config.log_every_steps=100\
--config.learning_rate=0.001  --config.optimizer={'adam'} --config.batch_size=128\
--config.num_para=7

x64 on
2023-07-25 14:28:37,648	ERROR utils.py:540 -- Unexpected error calculating docker cpuset ids.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/ray/_private/utils.py", line 537, in _get_docker_cpus
    cpu_ids.append(int(num_or_range))
ValueError: invalid literal for int() with base 10: '\n'
2023-07-25 14:28:38,749	INFO worker.py:1621 -- Started a local Ray instance.
2023-07-25 14:28:40,932	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
╭─────────────────────────────────────────────────────────────────────────────╮
│ Configuration for experiment     tune_with_parameters_2023-07-25_14-28-40   │
├─────────────────────────────────────────────────────────────────────────────┤
│ Search algorithm                 BasicVariantGener

In [ ]:
!python tuner_restore.py --workdir=/notebooks/GNN-ePC-SAFT/ --config=configs/default.py \
--restoredir=./ray/tune_with_parameters_2023-07-25_14-28-40 \
--config.num_train_steps=10000 --config.log_every_steps=100\
--config.learning_rate=0.001 --config.optimizer={'adam'} --config.batch_size=128\
--config.num_para=7

In [ ]:
!python evaluate.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7\
--config.checkpoint_every_steps=2000 --config.learning_rate=0.00001 --config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128\
--config.propagation_depth=5 --config.hidden_dim=512 --config.num_mlp_layers=2 --config.pre_layers=2 --config.post_layers=1